# Train MOTOR

This tutorial walks through the various steps to train a MOTOR model.

Training MOTOR is a four step process:

- Training a tokenizer
- Prefitting MOTOR
- Preparing batches
- Training the model

In [1]:
import shutil
import os

# os.environ["HF_DATASETS_CACHE"] = '/share/pi/nigam/zphuo/cache_dir'


TARGET_DIR = 'trash/tutorial_6_INSEPCT'

from_pretrained = False
num_proc = 20

In [2]:
if not from_pretrained:
    if os.path.exists(TARGET_DIR):
        shutil.rmtree(TARGET_DIR)

    os.mkdir(TARGET_DIR)
    os.mkdir(os.path.join(TARGET_DIR, 'motor_model'))

In [3]:
import datasets
import femr.index
import femr.splits

# First, we want to split our dataset into train, valid, and test
# We do this by calling our split functionality twice

# dataset = datasets.Dataset.from_parquet('input/meds/data/*')
parquet_folder = '/share/pi/nigam/projects/zphuo/data/PE/inspect/timelines_smallfiles_meds/data_subset/*'
dataset = datasets.Dataset.from_parquet(parquet_folder)


index = femr.index.PatientIndex(dataset, num_proc=num_proc)
main_split = femr.splits.generate_hash_split(index.get_patient_ids(), 97, frac_test=0.15)


# Note that we want to save this to the target directory since this is important information

main_split.save_to_csv(os.path.join(TARGET_DIR, "motor_model", "main_split.csv"))

train_split = femr.splits.generate_hash_split(main_split.train_patient_ids, 87, frac_test=0.15)

# print(train_split.train_patient_ids)
# print(train_split.test_patient_ids)

main_dataset = main_split.split_dataset(dataset, index)
train_dataset = train_split.split_dataset(main_dataset['train'], femr.index.PatientIndex(main_dataset['train'], num_proc=num_proc))

# print(train_dataset)

Map (num_proc=20):   0%|          | 0/1916 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/1639 [00:00<?, ? examples/s]

In [4]:
import femr.models.tokenizer
from femr.models.tokenizer import FEMRTokenizer
import pickle

# First, we need to train a tokenizer
# Note, we need to use a hierarchical tokenizer for MOTOR


with open('input/meds/ontology.pkl', 'rb') as f:
    ontology = pickle.load(f)

/share/pi/nigam/projects/zphuo/repos/transformers/src/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [5]:
if not from_pretrained:
    tokenizer = femr.models.tokenizer.train_tokenizer(
        main_dataset['train'], vocab_size=128, is_hierarchical=True, num_proc=num_proc, ontology=ontology)

    # Save the tokenizer to the same directory as the model
    tokenizer.save_pretrained(os.path.join(TARGET_DIR, "motor_model"))

else:
    # load pretrained tokenizer
    tokenizer = femr.models.tokenizer.FEMRTokenizer.from_pretrained(os.path.join(TARGET_DIR, "motor_model"), ontology=ontology)

Map (num_proc=20):   0%|          | 0/1639 [00:00<?, ? examples/s]

In [6]:
import femr.models.tasks

if 'subset' in parquet_folder:
    num_tasks = 39
else:
    num_tasks = 64

# Second, we need to prefit the MOTOR model. This is necessary because piecewise exponential models are unstable without an initial fit

motor_task = femr.models.tasks.MOTORTask.fit_pretraining_task_info(
    main_dataset['train'], tokenizer, num_tasks=num_tasks, num_bins=4, final_layer_size=32, num_proc=num_proc)


# It's recommended to save this with pickle to avoid recomputing since it's an expensive operation


Map (num_proc=20):   0%|          | 0/1639 [00:00<?, ? examples/s]

In [7]:
import femr.models.processor
import femr.models.tasks

# Third, we need to create batches. 

processor = femr.models.processor.FEMRBatchProcessor(tokenizer, motor_task)

# We can do this one patient at a time
# print("Convert a single patient")
# example_batch = processor.collate([processor.convert_patient(train_dataset['train'][0], tensor_type='pt')])

print("Convert batches")
# But generally we want to convert entire datasets
train_batches = processor.convert_dataset(train_dataset, tokens_per_batch=32, num_proc=num_proc)

print("Convert batches to pytorch")
# Convert our batches to pytorch tensors
train_batches.set_format("pt")
print("Done")

Convert batches


Map (num_proc=20):   0%|          | 0/1395 [00:00<?, ? examples/s]

Creating batches 340


Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=20):   0%|          | 0/244 [00:00<?, ? examples/s]

Creating batches 59


Generating train split: 0 examples [00:00, ? examples/s]

Convert batches to pytorch
Done


In [8]:
import transformers

# Finally, given the batches, we can train CLMBR.
# We can use huggingface's trainer to do this.

transformer_config = femr.models.transformer.FEMRTransformerConfig(
    vocab_size=tokenizer.vocab_size, 
    is_hierarchical=tokenizer.is_hierarchical, 
    n_layers=2,
    hidden_size=64, 
    intermediate_size=64*2,
    n_heads=8,
)

config = femr.models.transformer.FEMRModelConfig.from_transformer_task_configs(transformer_config, motor_task.get_task_config())

model = femr.models.transformer.FEMRModel(config)

collator = processor.collate

trainer_config = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,

    output_dir='tmp_trainer',
    remove_unused_columns=False,
    num_train_epochs=100,

    eval_steps=20,
    evaluation_strategy="steps",

    logging_steps=20,
    logging_strategy='steps',

    prediction_loss_only=True,
)

trainer = transformers.Trainer(
    model=model,
    data_collator=processor.collate,
    train_dataset=train_batches['train'],
    eval_dataset=train_batches['test'],
    args=trainer_config,
)


2024-02-02 19:37:54.788844: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-02 19:38:03.190238: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /share/pi/nigam/projects/zphuo/cuda/lib64::/home/zphuo/packages/cuda/lib64:/home/zphuo/miniconda3/lib:/home/zphuo/miniconda3/lib:/home/zphuo/miniconda3/bin
2024-02-02 19:38:03.190939: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such fi

In [9]:
trainer.train()

model.save_pretrained(os.path.join(TARGET_DIR, 'motor_model'))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Unable to read the token file at /var/run/secrets/kubernetes.io/serviceaccount/token due to permission error ([Errno 13] Permission denied: '/var/run/secrets/kubernetes.io/serviceaccount/token').The current user id is 1300074. Consider changing the securityContext to run the container as the current user.
wandb: WARNING Unable to read the token file at /var/run/secrets/kubernetes.io/serviceaccount/token due to permission error ([Errno 13] Permission denied: '/var/run/secrets/kubernetes.io/serviceaccount/token').The current user id is 1300074. Consider changing the securityContext to run the container as the current user.
wandb: Currently logged in as: zphuo (stanford_som). Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
20,0.432500,0.416286
40,0.445800,0.415241
60,0.442900,0.414404
80,0.456100,0.413633
100,0.489600,0.412867
120,0.429800,0.411946
140,0.392000,0.411218
160,0.384100,0.410018
180,0.420800,0.409019
200,0.417000,0.407995


Checkpoint destination directory tmp_trainer/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory tmp_trainer/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory tmp_trainer/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory tmp_trainer/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory tmp_trainer/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory tmp_trainer/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory tmp_trainer/checkpoint-3500 already exists and is non-empty. Saving will proceed

In [ ]:
# linear probe

